In [ ]:
from voc12 import VOC2012

In [1]:
!wget https://s3.amazonaws.com/fast-ai-imagelocal/pascal-voc.tgz 
!tar -xvzf ./pascal-voc.tgz

--2020-11-07 19:43:38--  https://s3.amazonaws.com/fast-ai-imagelocal/pascal-voc.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.48.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.48.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4639722845 (4.3G) [application/x-tar]
Saving to: ‘pascal-voc.tgz’

pascal-voc.tgz       87%[================>   ]   3.79G   522KB/s    in 3m 37s  


Cannot write to ‘pascal-voc.tgz’ (Success).
